In [1]:
import os
import sys
import networkx as nx
import pandas as pd


In [2]:
sys.path.append('../lib/')
import data_pre_processing
from models.gcn import GCN
sys.path.remove('../lib/')

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
A, X, Y, train_idx, val_idx, test_idx = data_pre_processing.load_data('cora', '../data/data')


(2708, 2708)
(2708, 1433)


In [4]:
import os,sys,inspect
import os
import joblib
import tensorflow as tf
import numpy as np
import h5py
import scipy.sparse.linalg as la
import scipy.sparse as sp
import scipy
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

import scipy.io as sio
import pdb
# 


In [5]:
#learning parameters and path dataset

num_total_iter_training = 3000
learning_rate = 1e-2
val_test_interval = 1
num_hidden_feat = 16
gamma = 5e-4

In [6]:
X = data_pre_processing.preprocess_features(X)

In [7]:
# compute GCN adj matrix
A_tilde = sp.csr_matrix(A)
A_tilde.setdiag(1)
D = A_tilde.sum(axis=1)

D_rows, D_cols = D.nonzero()
D_vals = [D[i,j] for i, j in zip(D_rows, D_cols)]
D_vals = np.reciprocal(np.sqrt(np.asarray(D_vals)))

D_inv_sqrt = sp.csr_matrix((D_vals, (range(len(D_vals)), range(len(D_vals)))))

A_tilde = D_inv_sqrt.dot(A_tilde).dot(D_inv_sqrt)
A_tilde = A_tilde.tocsr()
A_tilde.eliminate_zeros()

/Users/nikitaacharya/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
num_exp = 10 #number of times training GCN over the given dataset

list_all_acc = []
list_all_cost_val_avg  = []
list_all_data_cost_val_avg = []
list_all_acc_val_avg   = []
list_all_cost_test_avg = []
list_all_acc_test_avg  = []

num_done = 0
for seed in range(num_exp):
    GCNN = GCN(A_tilde, X, Y, num_hidden_feat, learning_rate=learning_rate, gamma=gamma)

    cost_train_avg      = []
    grad_norm_train_avg = []
    acc_train_avg       = []
    cost_test_avg       = []
    grad_norm_test_avg  = []
    acc_test_avg        = []
    cost_val_avg        = []
    data_cost_val_avg   = []
    acc_val_avg         = []
    iter_test           = []
    list_training_time = list()

    #Training code
    for i in range(num_total_iter_training):
        if (len(cost_train_avg) % val_test_interval) == 0:
            #Print last training performance
            if (len(cost_train_avg)>0):
                print("[TRN] epoch = %03i, cost = %3.2e, |grad| = %.2e, acc = %3.2e (%03.2fs)" % \
                (len(cost_train_avg), cost_train_avg[-1], grad_norm_train_avg[-1], acc_train_avg[-1], time.time() - tic))

            #Validate the model
            tic = time.time()
            
            feed_dict = {GCNN.idx_nodes: val_idx, GCNN.keep_prob:1.0}
            acc_val, cost_val, data_cost_val = GCNN.session.run([GCNN.accuracy, GCNN.loss, GCNN.data_loss], feed_dict)
            
            data_cost_val_avg.append(data_cost_val)
            cost_val_avg.append(cost_val)
            acc_val_avg.append(acc_val)
            print("[VAL] epoch = %03i, data_cost = %3.2e, cost = %3.2e, acc = %3.2e (%03.2fs)" % \
                (len(cost_train_avg), data_cost_val_avg[-1], cost_val_avg[-1], acc_val_avg[-1],  time.time() - tic))

            #Test the model
            tic = time.time()
            
            feed_dict = {GCNN.idx_nodes: test_idx, GCNN.keep_prob:1.0}
            acc_test, cost_test = GCNN.session.run([GCNN.accuracy, GCNN.loss], feed_dict)
            
            cost_test_avg.append(cost_test)
            acc_test_avg.append(acc_test)
            print("[TST] epoch = %03i, cost = %3.2e, acc = %3.2e (%03.2fs)" % \
                (len(cost_train_avg), cost_test_avg[-1], acc_test_avg[-1],  time.time() - tic))
            iter_test.append(len(cost_train_avg))

        tic = time.time()

        tic = time.time()
        feed_dict = {GCNN.idx_nodes: train_idx, GCNN.keep_prob: 0.5}
        
        _, current_training_loss, norm_grad, current_acc_training = GCNN.session.run([GCNN.opt_step, GCNN.loss, GCNN.norm_grad, GCNN.accuracy], feed_dict) 

        training_time = time.time() - tic   

        cost_train_avg.append(current_training_loss)
        grad_norm_train_avg.append(norm_grad)
        acc_train_avg.append(current_acc_training)

    #Compute and print statistics of the last realized experiment
    list_all_acc.append(100*(np.asarray(acc_test_avg)[np.asarray(data_cost_val_avg)==np.min(data_cost_val_avg)]))
    list_all_cost_val_avg.append(cost_val_avg)
    list_all_data_cost_val_avg.append(data_cost_val_avg)
    list_all_acc_val_avg.append(acc_val_avg)
    list_all_cost_test_avg.append(cost_test_avg)
    list_all_acc_test_avg.append(acc_test_avg)

    print('Num done: %d' % num_done)
    print('Max accuracy on test set achieved: %f%%' % np.max(np.asarray(acc_test_avg)*100))
    print('Max suggested accuracy: %f%%' % (100*(np.asarray(acc_test_avg)[np.asarray(data_cost_val_avg)==np.min(data_cost_val_avg)]),))
    print('Current mean: %f%%' % np.mean(list_all_acc))
    print('Current std: %f' % np.std(list_all_acc))

    num_done += 1

AttributeError: module 'tensorflow.python.keras.api._v1.keras.layers' has no attribute 'xavier_initializer'